In [2]:
myf = open('/data/private/E2E/predictions/final/final_model.txt')
tgenf = open('/data/private/E2E/compared_system/tgen.txt')
slugf = open('/data/private/E2E/compared_system/tgen.txt')

our = myf.readlines()
tgen = tgenf.readlines()
slug = slugf.readlines()

myf.close()
tgenf.close()
slugf.close()

In [16]:
import pandas as pd
testset = pd.read_csv('/data/private/E2E/dataset/testset.csv')

In [17]:
testset.columns

Index(['MR'], dtype='object')

In [18]:
testset['MR'], len(testset), len(slug)

(0      name[Blue Spice], eatType[coffee shop], area[c...
 1      name[Blue Spice], eatType[coffee shop], area[r...
 2      name[Blue Spice], eatType[coffee shop], custom...
 3      name[Blue Spice], eatType[coffee shop], custom...
 4      name[Blue Spice], eatType[coffee shop], custom...
                              ...                        
 625    name[Zizzi], eatType[coffee shop], near[The So...
 626           name[Zizzi], eatType[pub], area[riverside]
 627    name[Zizzi], eatType[pub], customer rating[ave...
 628    name[Zizzi], eatType[pub], customer rating[hig...
 629        name[Zizzi], eatType[pub], near[The Sorrento]
 Name: MR, Length: 630, dtype: object,
 630,
 630)

In [23]:
sample_num = 100

import random, csv
idx_list = [x for x in range(630)]
sample_list = random.sample(idx_list, sample_num)

f = open('input.csv','w', newline='')
wr = csv.writer(f)
wr.writerow(['MRs','text', 'model'])

for i in range(sample_num):
    idx = sample_list[i]
    
    MRs = testset['MR'][idx]
    
    our_sentence = our[idx].strip()
    tgen_sentence = tgen[idx].strip()
    slug_sentence = slug[idx].strip()
    
    wr.writerow([MRs, our_sentence, 'ours'])
    wr.writerow([MRs, tgen_sentence, 'tgen'])
    wr.writerow([MRs, slug_sentence, 'slug'])
    
f.close()

In [33]:
testset['MR'][250]

'name[The Mill], eatType[restaurant], food[Fast food], priceRange[cheap], area[riverside], familyFriendly[yes], near[Raja Indian Cuisine]'

In [25]:
our0[0], style0[0]

("ever since joes has changed hands it 's just gotten worse and worse .\thowever since joes has hands it 's just gotten great and delicious .\t0\n",
 "ever since joes has changed hands it 's just gotten worse and worse .\ti carne asada does made every only it just the prices are get mediocre stuff .\t0\n")

In [21]:
random.randrange(0,1)

0

## batch

In [126]:
import pandas as pd
batch = pd.read_csv('/data/private/E2E/AMT/batch.csv')

In [127]:
batch.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.MRs',
       'Input.text', 'Input.model', 'Answer.contain_MR', 'Answer.naturalness',
       'Answer.quality', 'Approve', 'Reject'],
      dtype='object')

In [128]:
limit = 0

model_list = ['ours', 'tgen', 'slug']
contain_MR = [0, 0, 0]
naturalness = [0, 0, 0]
quality = [0, 0, 0]
count = [0, 0, 0]

for i in range(len(batch['Input.model'])):
    if str(batch['RequesterFeedback'][i]) != 'nan' or batch['WorkTimeInSeconds'][i] < limit:
        continue
    model = batch['Input.model'][i]
    
    contain_MR_score = batch['Answer.contain_MR'][i]
    naturalness_score = batch['Answer.naturalness'][i]
    quality_score = batch['Answer.quality'][i]
    
    k = model_list.index(model)
    count[k] += 1
    contain_MR[k] += contain_MR_score
    naturalness[k] += naturalness_score
    quality[k] += quality_score    

for k in range(len(count)):
    contain_MR[k] /= count[k]
    naturalness[k] /= count[k]
    quality[k] /= count[k]       

In [129]:
print('model ##  ', 'conatin_MR ', 'naturalness ', 'quality')
for i in range(len(model_list)):
    model = model_list[i]
    print(model,' ## ', contain_MR[i], naturalness[i], quality[i])

model ##   conatin_MR  naturalness  quality
ours  ##  3.91 3.8 3.82
tgen  ##  3.9833333333333334 3.8333333333333335 3.8633333333333333
slug  ##  3.98 3.8366666666666664 3.8466666666666667


In [130]:
count, sum(count)

([300, 300, 300], 900)

In [147]:
limit = 11

model_list = ['ours', 'tgen', 'slug']
contain_MR = [[], [], []]
naturalness = [[], [], []]
quality = [[], [], []]
count = [0, 0, 0]

for i in range(len(batch['Input.model'])):
    if str(batch['RequesterFeedback'][i]) != 'nan' or batch['WorkTimeInSeconds'][i] < limit:
        continue
    model = batch['Input.model'][i]
    
    contain_MR_score = batch['Answer.contain_MR'][i]
    naturalness_score = batch['Answer.naturalness'][i]
    quality_score = batch['Answer.quality'][i]
    
    k = model_list.index(model)
    count[k] += 1
    contain_MR[k].append(contain_MR_score)
    naturalness[k].append(naturalness_score)
    quality[k].append(quality_score)

num = 200
c_out = [0,0,0]
n_out = [0,0,0]
q_out = [0,0,0]


for k in range(len(count)):    
    ord_list = [i[0] for i in sorted(enumerate(contain_MR[k]), key=lambda x:x[1])]
    for x in range(num):
        c_out[k] += contain_MR[k][ord_list[x]]
        n_out[k] += naturalness[k][ord_list[x]]
        q_out[k] += quality[k][ord_list[x]]
    c_out[k] /= num
    n_out[k] /= num
    q_out[k] /= num
        
        
#     contain_MR[k].sort(reverse=True)
#     naturalness[k].sort(reverse=True)
#     quality[k].sort(reverse=True)
    
#     c_out[k] = sum(contain_MR[k][:num])/num
#     n_out[k] = sum(naturalness[k][:num])/num
#     q_out[k] = sum(quality[k][:num])/num

# for k in range(len(count)):    
#     contain_MR[k].sort(reverse=True)
#     naturalness[k].sort(reverse=True)
#     quality[k].sort(reverse=True)
    
#     c_out[k] = sum(contain_MR[k][:num])/num
#     n_out[k] = sum(naturalness[k][:num])/num
#     q_out[k] = sum(quality[k][:num])/num

In [148]:
print('model ##  ', 'contain_MR ', 'naturalness ', 'quality')
for i in range(len(model_list)):
    model = model_list[i]
    print(model,' ## ', c_out[i], n_out[i], q_out[i])

model ##   contain_MR  naturalness  quality
ours  ##  3.545 3.56 3.55
tgen  ##  3.625 3.625 3.6
slug  ##  3.61 3.57 3.54


In [133]:
sum(contain_MR[k][:num])

800

## bathc_1 final (210)

In [171]:
limit = 0

model_list = ['ours', 'tgen', 'slug']
contain_MR = [0, 0, 0]
naturalness = [0, 0, 0]
quality = [0, 0, 0]
count = [0, 0, 0]

num = 70
for i in range(num*9):
    if str(batch['RequesterFeedback'][i]) != 'nan' or batch['WorkTimeInSeconds'][i] < limit:
        continue
    model = batch['Input.model'][i]
    
    contain_MR_score = batch['Answer.contain_MR'][i]
    naturalness_score = batch['Answer.naturalness'][i]
    quality_score = batch['Answer.quality'][i]
    
    k = model_list.index(model)
    count[k] += 1
    contain_MR[k] += contain_MR_score
    naturalness[k] += naturalness_score
    quality[k] += quality_score    

for k in range(len(count)):
    contain_MR[k] /= count[k]
    naturalness[k] /= count[k]
    quality[k] /= count[k]       

In [172]:
print('model ##  ', 'conatin_MR ', 'naturalness ', 'quality')
for i in range(len(model_list)):
    model = model_list[i]
    print(model,' ## ', contain_MR[i], naturalness[i], quality[i])

model ##   conatin_MR  naturalness  quality
ours  ##  3.9571428571428573 3.8333333333333335 3.8095238095238093
tgen  ##  4.0095238095238095 3.8285714285714287 3.880952380952381
slug  ##  3.9761904761904763 3.8047619047619046 3.8047619047619046


In [173]:
count, sum(count)

([210, 210, 210], 630)

## batch_2

In [9]:
import pandas as pd
batch = pd.read_csv('/data/private/E2E/AMT/final_batch.csv')
# batch = pd.read_csv('/data/private/E2E/AMT/pre/eval_batch.csv')

In [10]:
limit = 0

model_list = ['ours', 'tgen', 'slug']
naturalness = [0, 0, 0]
quality = [0, 0, 0]
count = [0, 0, 0]

for i in range(len(batch['Input.model'])):
    if str(batch['RequesterFeedback'][i]) != 'nan' or batch['WorkTimeInSeconds'][i] < limit:
        continue
    model = batch['Input.model'][i]
    
    naturalness_score = batch['Answer.naturalness'][i]
    quality_score = batch['Answer.quality'][i]
    
    k = model_list.index(model)
    count[k] += 1
    naturalness[k] += naturalness_score
    quality[k] += quality_score    

for k in range(len(count)):
    naturalness[k] /= count[k]
    quality[k] /= count[k]       

In [11]:
print('model ##  ', 'naturalness ', 'quality')
for i in range(len(model_list)):
    model = model_list[i]
    print(model,' ## ', naturalness[i], quality[i])

model ##   naturalness  quality
ours  ##  4.2207357859531776 4.260869565217392
tgen  ##  4.126666666666667 4.1433333333333335
slug  ##  4.160535117056856 4.110367892976589


In [12]:
count, sum(count)

([299, 300, 299], 898)

In [ ]:
final_batch ([299, 300, 299], 898)
model ##   naturalness  quality
ours  ##  4.2207357859531776 4.260869565217392
tgen  ##  4.126666666666667 4.1433333333333335
slug  ##  4.160535117056856 4.110367892976589

In [ ]:
batch 6 ([200, 205, 200], 605) // eval_batch
model ##   naturalness  quality
ours  ##  4.625 4.525
tgen  ##  4.4975609756097565 4.317073170731708
slug  ##  4.545 4.34

In [ ]:
batch2
model ##   naturalness  quality
ours  ##  4.620689655172414 4.489655172413793
tgen  ##  4.515873015873016 4.301587301587301
slug  ##  4.548148148148148 4.303703703703704

In [ ]:
batch 3
model ##   naturalness  quality
ours  ##  4.619631901840491 4.5030674846625764
tgen  ##  4.512820512820513 4.269230769230769
slug  ##  4.566265060240964 4.319277108433735

In [ ]:
batch 4
model ##   naturalness  quality
ours  ##  4.620689655172414 4.494252873563219
tgen  ##  4.483333333333333 4.272222222222222
slug  ##  4.54696132596685 4.30939226519337

In [ ]:
batch 5
model ##   naturalness  quality
ours  ##  4.62962962962963 4.518518518518518
tgen  ##  4.481675392670157 4.277486910994765
slug  ##  4.526315789473684 4.3052631578947365